# Artificial Neural Network Model

### IF3170 - Machine Learning

Developed by: 
1. Juan Christopher Santoso (13521116)
2. Nicholas Liem (13521135)
3. Nathania Calista Djunaedi (13521139)
4. Antonio Natthan Krishna (13521162)

#### **Import Library**

In [217]:
import numpy as np
import json
import copy
import pickle

#### **Neural Network Properties**

#### 1. Activation Function

In [218]:
class ActivationFunction:
    def __init__(self, types='Sigmoid'):
        self.func = self.sigmoid
        self.dfunc = self.dsigmoid
        self.dfuncerr = self.dsum_square

        match types:
            case 'sigmoid':
                self.func = self.sigmoid
                self.dfunc = self.dsigmoid
                self.dfuncerr = self.dsum_square
            case 'linear':
                self.func = self.linear
                self.dfunc = self.dlinear
                self.dfuncerr = self.dsum_square
            case 'softmax':
                self.func = self.softmax
                self.dfuncerr = self.derr_softmax
            case 'relu':
                self.func = self.relu
                self.dfunc = self.drelu
                self.dfuncerr = self.dsum_square

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def dsigmoid(self, x):
        sig = self.sigmoid(x)
        return sig * (1-sig)
    
    def linear(self, x):
        return x
    
    def dlinear(self, x):
        return 1
    
    def softmax(self, x):
        expX = np.exp(x - np.max(x, axis=1, keepdims=True))
        return expX / np.sum(expX, axis=1, keepdims=True)
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def drelu(self, x):
        return np.where(x > 0, 1, 0)
    
    def dsum_square(self, output, target):
        return target - output
    
    def derr_softmax(self, output, target):
        if (target != 1):
            return output
        else:
            return output - 1

#### 2. Loss Function

In [219]:
class LossFunction:
    @staticmethod
    def calculate(output, target, layers):
        activation_mode = layers[-1]['activation_function']
        if (activation_mode == "softmax"):
            return LossFunction.loss_softmax(output, target)
        else:
            return LossFunction.loss_rsl(output, target)

    @staticmethod
    def loss_rsl(output, target):
        err = 0
        for i in range (len(output)):
            err += (target[i] - output[i])**2
        return err
    
    @staticmethod
    def loss_softmax(output, target):
        idx = np.argmax(target)
        return -1 * np.log10(output[idx])

#### 3. Forward Propagation (Using Fast Forward Neural Network)

In [220]:
class ForwardPropagation:
    @staticmethod
    def process(input_data, layers, weights):
        activations = input_data
        neuron_net = []
        neuron_out = []

        for i in range(len(layers)):
            activations_with_bias = np.insert(activations, 0, 1, axis=1)
            net_input = np.dot(activations_with_bias, weights[i])
            activation_mode = layers[i]['activation_function']
            activationFunc = ActivationFunction(activation_mode)
            activations = activationFunc.func(net_input)
            neuron_net.append(net_input)
            neuron_out.append(activations)

        return activations, neuron_net, neuron_out

#### 4. Backward Propagation

In [221]:
class BackwardPropagation:
    @staticmethod
    def process(weights, output, target, neuron_out, layers, learning_rate, input_data):
        delta = []

        # Delta Output Layer
        activation_mode = layers[-1]['activation_function']
        delta_layer = []
        activationFunc = ActivationFunction(activation_mode)
        if (activation_mode == 'softmax'):
            for i in range (len(output)): 
                delta_layer.append(activationFunc.dfuncerr(output[i], target[i]))
        else:
            for i in range (len(output)): 
                delta_layer.append(activationFunc.dfuncerr(output[i], target[i]) * activationFunc.dfunc(output[i]))
        delta.append(delta_layer)

        # Delta Hidden Layer
        for i in range(len(layers) - 2, -1, -1):
            activation_mode = layers[i]['activation_function']
            activationFunc = ActivationFunction(activation_mode)
            prev_delta = delta_layer if i == len(layers) - 2 else delta_layer[1:]
            delta_layer = []
            layer_weight = weights[i+1]
            layer_output = neuron_out[i][0]

            for j in range (len(layer_weight)): 
                neuron_weight = layer_weight[j]
                sigma = np.dot(neuron_weight, prev_delta)
                assert activation_mode != "softmax", "Softmax cannot be in hidden layers"
                delta_layer.append(sigma * activationFunc.dfunc(1 if j == 0 else layer_output[j-1]))
            delta = [delta_layer[1:]] + delta

        # Update Weight
        for i in range (len(layers)):
            if (i == 0):
                layer_input = input_data
            else :
                layer_input = neuron_out[i-1][0]

            layer_input = np.insert(layer_input, 0, 1)
            for j in range (len(weights[i])):
                for k in range (len(weights[i][j])):
                    weights[i][j][k] += learning_rate * delta[i][k] * layer_input[j]
        return weights

#### **Artificial Neural Network**

In [222]:
class ArtificialNeuralNetwork:
    def __init__(self, architecture):
        self.layers = architecture.layers
        self.learning_rate = architecture.learning_rate
        self.error_threshold = architecture.error_threshold
        self.max_iter = architecture.max_iteration
        self.batch_size = architecture.batch_size
        self.input_data = architecture.input
        self.target = architecture.target
        self.weights = architecture.initial_weights

    def save_weights(self, file_path):
      with open(file_path, 'wb') as file:
        pickle.dump(self.weights, file)

    def load_weights(self, file_path):
      with open(file_path, 'rb') as file:
        self.weights = pickle.load(file)

    def train(self):
        temp_weight = copy.deepcopy(self.weights)

        for i in range (self.max_iter):
            error_total = 0
            minibatch = self.batch_size
            for i in range (len(self.input_data)):
                if (minibatch == 0):
                    self.weights = copy.deepcopy(temp_weight)
                    minibatch = self.batch_size
                output, _, neuron_out = ForwardPropagation.process([self.input_data[i]], self.layers, self.weights)
                error_total += LossFunction.calculate(output[0], self.target[i], self.layers)
                temp_weight = BackwardPropagation.process(temp_weight, output[0], self.target[i], neuron_out, self.layers, self.learning_rate, self.input_data[i])
                minibatch = minibatch - 1
            
            self.weights = copy.deepcopy(temp_weight)
            if (error_total < self.error_threshold):
                break
        

        for weight_group in self.weights:
            print("[")
            for weight in weight_group:
                print("\t", weight)
            print("]")
        
        print("\nTOTAL ERROR VAL:", error_total)


#### **Preprocessing Data**

In [223]:
class JsonModelParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self.load_json_file()
        self.parse_model_data()

    def printDetails(self):
        print("\tINPUT SIZE:",self.input_size)
        print("\tLAYERS:", self.layers)
        print("\tINPUT:", self.input)
        print("\tINITIAL WEIGHTS:", self.initial_weights)
        print("\tTARGET:", self.target)
        print("\tLEARNING RATE:", self.learning_rate)
        print("\tBATCH SIZE:", self.batch_size)
        print("\tMAX ITERATION:", self.max_iteration)
        print("\tERROR THRESHOLD:", self.error_threshold)
    

    def load_json_file(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"The file {self.filepath} was not found")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {self.filepath}")
            return None

    def parse_model_data(self):
        if self.data:
            self.case = self.data.get('case', {})
            self.model = self.case.get('model', {})
            self.input_size = self.model.get('input_size')

            raw_layers = self.model.get('layers', [])   
            self.layers = [{'number_of_neurons': layer.get('number_of_neurons'),
                        'activation_function': layer.get('activation_function')}
                       for layer in raw_layers]
            
            self.input = self.case.get('input', [])
            self.initial_weights = self.case.get('initial_weights', [])
            self.target = self.case.get('target', [])
            self.parameters = self.case.get('learning_parameters', {})

            self.learning_rate = self.parameters.get('learning_rate')
            self.batch_size = self.parameters.get('batch_size')
            self.max_iteration = self.parameters.get('max_iteration')
            self.error_threshold = self.parameters.get('error_threshold')

            self.expect = self.data.get('expect', {})
            self.stopped_by = self.expect.get('stopped_by', '')
            self.final_weights = self.expect.get('final_weights', [])

    @staticmethod
    def save_json_file(data, filepath):
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
        except IOError:
            print(f"Could not save data to {filepath}")

#### **Processing Data**

In [224]:
def sum_squared_error(weights1, weights2):
    sse = 0
    for layer1, layer2 in zip(weights1, weights2):
        for neuron_weights1, neuron_weights2 in zip(layer1, layer2):
            for weight1, weight2 in zip(neuron_weights1, neuron_weights2):
                diff = weight1 - weight2
                sse += diff * diff
    return sse

In [225]:

# Read all files inside the test folders
import os

currDir = os.getcwd()
testDir = currDir.replace("src", "test")

testFiles = os.listdir(testDir)
testFilesArr = []

for filename in testFiles:
  if (".json" in filename):
    testFilesArr.append(os.path.join(testDir, filename))

# Use every file inside testFilesArr as input

for testFile in testFilesArr:
  print("FILENAME:", testFile.split("\\")[-1])
  architecture = JsonModelParser(testFile)
  architecture.printDetails()
  print("\n")
  print("RESULT:")
  model = ArtificialNeuralNetwork(architecture)
  model.train()

  error = sum_squared_error(model.weights, architecture.final_weights)
  print("SSE (SUM SQUARED ERROR):", error)

  print("====================================================================")
  print("====================================================================")
  print("\n")

FILENAME: /home/nicholas/Desktop/Repository/Personal/IF3270_BP_ANN/test/softmax.json
	INPUT SIZE: 8
	LAYERS: [{'number_of_neurons': 3, 'activation_function': 'softmax'}]
	INPUT: [[-2.4, -2.78, -0.6, 0.37, 2.46, -0.92, 2.76, 2.62], [-1.79, 1.65, -0.77, -1.03, 0.1, 2.12, -2.36, 1.25], [1.65, 2.34, 0.27, 2.34, 0.52, 1.37, 1.77, 0.62]]
	INITIAL WEIGHTS: [[[0.1, 0.9, -0.1], [-0.2, 0.8, 0.2], [0.3, -0.7, 0.3], [0.4, 0.6, -0.4], [0.5, 0.5, 0.5], [-0.6, 0.4, 0.6], [-0.7, -0.3, 0.7], [0.8, 0.2, -0.8], [0.9, -0.1, 0.0]]]
	TARGET: [[0, 1, 0], [1, 0, 0], [0, 0, 1]]
	LEARNING RATE: 0.01
	BATCH SIZE: 1
	MAX ITERATION: 10
	ERROR THRESHOLD: 0.05


RESULT:
[
	 [0.14904856347734866, 0.8401686383772108, -0.08921720185455931]
	 [-0.15706354161481173, 1.0830768393090113, -0.12601329769419944]
	 [0.001273263187634495, -0.35718015754349425, 0.25590689435585984]
	 [0.43515106091312644, 0.6658486569253259, -0.5009997178384523]
	 [0.765771885554136, 0.545636460733514, 0.18859165371235018]
	 [-0.3497496464031748